# Student Loan Interest Calculator + Visualization
Runs various studies on loans to determine the optimal means for paying off the loan.

Example Loans:
- Loan Manager: FedLoanServicing
- Loan 1: 6507.64, 6.410% APR
- Loan 2: 21263.06, 5.410% APR
- Loan 3: 6994.65, 7.900% APR
- Loan 4: 23562.73, 6.800% APR

The following events will trigger interest recapitalization:
1. When the loan enters repayment.
2. When a deferment ends.
3. When forbearance ends.
4. When the loan defaults.
5. A change in repayment plan.
6. Loan consolidation. 

Main source of information: https://studentaid.ed.gov

Definition of Terms
1. <b>Enters repayment</b>: you leave school and now must pay your loans back.
2. <b>Deferment Ends</b>: A deferment occurs when you stop making payments on your student loans. It has special rules about when you have to pay the interest on your loans during a deferment.
 - During a deferment, you generally don't have to pay for interest if you have a subsidized direct, federal stafford, direct consolodated, or FFEL consolidated loans. This is because whoever subsidizes your loan is paying the interest as it occurs.
 - You <b>must</b> pay interest on all subsidized loans.
 - Deferment is basically the default status of a student loan while you are in school, and generally, if you don't pay the accrued interest on unsubsidized loans, this unpaid interest gets added to your principal loan balance, which then increases the total interest payment, because now you effectively have a larger loan.
3. <b>Forbearance Ends</b>:  
 - Forbearance is essentially the same thing as a deferrment in that you tell your loan servicer that you cannot make payments against your principal balance. The difference is that you are always responsible for paying the interest during a forbearance. After forbearance ends, unpaid interest is recapitalized.
 - Forbearance and Deferment both free you from making loan payments against your principal, but you're just burning money as interest.
4. <b>Default</b>: Student loan default when you fail to make loan payments as scheduled.
 - When you default on your loan, legal action can be taken to force you, or the person who cosigned your loan (a guarantor), or both to pay the balance of the loan. This can happen in a broad variety of very unpleasant ways, such as wage garnishing, repossession of property, etc.
 - Usually, prior to going into default, you enter a period of <b>delinquency</b> which means you haven't made the loan payments you agreed to when you signed your promisary note upon applying for the loan in the first place. Being delinquent has bad consequences too, it effects your credit score, and can prevent you from entering into financial arrangements with entities, such as getting other loans, getting approval to rent an apartment, getting homeowners' insurance, or even signing up for utilities.
 - If you pay your loan monthly, default occurs after missing a payment for 270 days.
 - There is no guaranteed way to exit default, you have to contact your loan servicer or collections agency.
 - <b> There are huge consequences to going into default. Its really bad. </b>
 	- You must pay the entire balance of your loans immediately (obviously impossible), you lose elgibility for deferment or forbearance, you lose elgiability for all federal student loans, your loan may go to collections, your credit score tanks, your federal tax return may be given to the debt collectors, your total debt will balloon with fees and unpaid interest, your wages can be garnished, you may be sued and blocked from purchasing or selling certain assets (such as a house), federal employees may sacrifice 15% of pay, and you must spend years to re-establish your credit rating.
  
  

# Analysis
Loans are tracked based on inital and current state. This allows us to essentially simulate different payment strategies and how this affects stuff like interest paid, payoff time, etc. Another interesting approach might be to vectorize stuff to allow for more powerful solver methods.

In [3]:
# TODO - fix importing the loan_solver package
loans = [
    {
        'loan_name':'loan_641',
        'loan_type':'StudentLoan',
        'principal_balance':6507.64,
        'interest_rate':0.0641
    },
    {
        'loan_name':'loan_541',
        'loan_type':'StudentLoan',
        'principal_balance':21263.06,
        'interest_rate':0.0541,
    },
    {
        'loan_name':'loan_790',
        'loan_type':'StudentLoan',
        'principal_balance':6994.65,
        'interest_rate':0.068,
    },
    {
        'loan_name':'loan_680',
        'loan_type':'StudentLoan',
        'principal_balance':23562.73,
        'interest_rate':0.068,
    },
    
]

def always_pay_highest_balance(payment, time_unit='month', loans=loans):
    print("Calculating Pay According to Balance - Payment is split according to balance distribution")
    solver = LoanSolver()
    for loan in loans:
        solver.add_loan(**loan)
    done = False
    time = 0
    # Pay According To Loan Amount
    while not done:
        solver.add_interest(1,time_unit)
        time += 1
        distribution = solver.get_balance_distribution()
        # TODO - fix how remainder is applied, if any (returns from pay_loan)
        for loan_name in distribution:
            solver.pay_loan(loan_name, payment*distribution[loan_name])
        if solver.debt_free():
            done = True
            print("\tTotal Paid {}, Principal {}, Total Interest {}, in {} {}s".format(solver.total_paid, solver.get_initial_principal(), solver.total_paid - solver.get_initial_debt(), time, time_unit))
     

def always_pay_highest_interest(payment, time_unit='month', loans=loans):
    # Pay According to Interest Amount
    print("Calculating Pay According to Interest - Payment is split according to the interest distribution")
    solver = LoanSolver()
    for loan in loans:
        solver.add_loan(**loan)
    time = 0
    done = False
    while not done:
        solver.add_interest(1,time_unit)
        distribution = solver.get_interest_distribution()
        time += 1
        for loan_name in distribution:
            solver.pay_loan(loan_name, payment*distribution[loan_name])
        if solver.debt_free():
            done = True
            print("\tTotal Paid {}, Principal {}, Total Interest {}, in {} {}s ".format(solver.total_paid, solver.get_initial_principal(), solver.total_paid - solver.get_initial_debt(), time, time_unit))

def pay_snowball(paymentt, time_unit='month', loans=loans):
    # Snowball
    print("Calculating Snowball Method")
    solver = LoanSolver()
    for loan in loans:
        solver.add_loan(**loan)
    time = 0
    done = False
    loan_name = solver.smallest_principal()
    while not done:
        solver.add_interest(1,time_unit)
        payment = 1500
        time += 1
        solver.pay_loan(loan_name, payment)
        if loan_name not in solver.loans:
            # a loan is paid off
            loan_name = solver.smallest_principal()
            if loan_name is None:
                done = True
                print("\tTotal Paid {}, Principal {}, Total Interest {}, in {} {}s ".format(solver.total_paid, solver.get_initial_principal(), solver.total_paid - solver.get_initial_debt(), time, time_unit))
            
        
def pay_avalanche(payment, time_unit='month', loans=loans):
    # Avalanche
    # TODO - pick the largest loan and pay off entirely, not continuously the largest balance
    print("Calculating Avalanche Method")
    solver = LoanSolver()
    for loan in loans:
        solver.add_loan(**loan)
    time = 0
    done = False
    loan_name = solver.largest_principal()
    while not done:
        solver.add_interest(1,time_unit)
        time += 1
        solver.pay_loan(loan_name, payment)
        if loan_name not in solver.loans:
            # a loan is paid off
            loan_name = solver.largest_principal()
            if loan_name is None:
                done = True
                print("\tTotal Paid {}, Principal {}, Total Interest {}, in {} {}s ".format(solver.total_paid, solver.get_initial_principal(), solver.total_paid - solver.get_initial_debt(), time, time_unit))

ModuleNotFoundError: No module named 'loan_solver.LoanSolver'

In [ ]:
payment = 3000
always_pay_highest_balance(payment)
always_pay_highest_interest(payment)
pay_snowball(payment)
pay_avalanche(payment)